In [1]:
import numpy as np
import pandas as pd
import GaussianCopula as gc
import os
from sklearn.model_selection import train_test_split
import copy
import utils
import matplotlib.pyplot as plt
import operator

In [2]:
# Note that we assume the last column is the label
# If the first column is ID's, remove it
filename = "pulsar_stars.csv"
df = pd.read_csv("Datasets/"+filename)
colnames = [('X'+str(i)) for i in range(df.shape[1]-1)]
colnames.append('Y')
cont_col = [i for i in range(8)]
df.columns = colnames

In [3]:
X = df.loc[:5000,colnames[:-1]].to_numpy()
Y = df.loc[:5000,colnames[-1]].to_numpy()
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
cont_col = [0,1,2,3,4,5,6,7]

In [4]:
# copula classifier
clf = gc.CopulaClassifier(cont_col, None, None, 0.9)
clf.fit(x_train,y_train)
prob_distribution = clf.test_prob_dist(x_test)
prediction = {i: max(prob_distribution[i].items(), key=operator.itemgetter(1))[0] for i in prob_distribution.keys()}
utils.get_accuracy(prediction, y_test)

Finished clustering
Optimized bandwidth


/Users/pengcheng/Desktop/FYP/Improvement-to-Naive-Bayes-Classifier/utils.py:160: RuntimeWarning: invalid value encountered in matmul
  b = np.matmul(vec, mid_mat)
/Users/pengcheng/Desktop/FYP/Improvement-to-Naive-Bayes-Classifier/utils.py:161: RuntimeWarning: invalid value encountered in matmul
  c = np.matmul(b, vec.T)


0.8502793296089386

In [19]:
# Gaussian Naive Bayes
from sklearn.naive_bayes import GaussianNB
clf2 = GaussianNB()
clf2.fit(x_train,y_train)
nb_predict = clf2.predict(x_test)
utils.get_accuracy(nb_predict,y_test)

0.9440559440559441

In [4]:
# Clustered comonotonic
import comonotonic as cm
clf3 = cm.clustered_comonotonic(x_train,y_train,None,cont_col,None,0.8)
clf3.run_cluster()
cm_predict = clf3.cluster_predict(x_test)
utils.get_accuracy(cm_predict, y_test)

0.954045954045954

In [5]:
clf4 = cm.pure_comonotonic(x_train,y_train,None,cont_col,None)
clf4.run()
clf4_predict = clf4.predict(x_test)
utils.get_accuracy(clf4_predict,y_test)

0.7932067932067932

In [10]:
clf4.bin_info

{0: array([ 24.54951721,  52.55398283,  80.55844844, 108.56291406,
        136.56737968, 164.5718453 , 192.57631092]),
 1: array([24.28685748, 31.58394379, 38.8810301 , 46.17811642, 53.47520273,
        60.77228904, 68.06937535]),
 2: array([-3.08545657, -1.85910806, -0.63275954,  0.59358897,  1.81993748,
         3.04628599,  4.2726345 ]),
 3: array([-19.35324725, -12.11114156,  -4.86903587,   2.37306982,
          9.61517551,  16.8572812 ,  24.0993869 ]),
 4: array([-68.39150405, -41.77388133, -15.1562586 ,  11.46136413,
         38.07898686,  64.69660959,  91.31423231]),
 5: array([-27.42397712,  -9.54442759,   8.33512195,  26.21467148,
         44.09422102,  61.97377056,  79.85332009]),
 6: array([-4.75286594, -0.47339311,  3.80607972,  8.08555254, 12.36502537,
        16.6444982 , 20.92397103]),
 7: array([-200.06773594, -100.63709875,   -1.20646156,   98.22417563,
         197.65481282,  297.08545002,  396.51608721])}

In [13]:
checker = gc.CopulaClassifier(cont_col, None, None, 0.9, use_default_bandwidth=False)

In [14]:
checker.x_train = x_train
checker.classes = np.sort(np.unique(y_train)).astype(int)
checker.divided_train_set = [x_train[y_train == yi] for yi in checker.classes] # divide the training set by classes
checker.prior_prob = {checker.classes[i]:np.log(checker.divided_train_set[i].shape[0]/x_train.shape[0]) for i in range(len(checker.classes))}
checker.rankable = [i for i in range(len(checker.x_train[0]))]

In [15]:
checker.optimize_bandwidth()

Start to find opt bandwidth for class, feature0,0
Found opt bandwidth for class, feature0,0
Start to find opt bandwidth for class, feature0,1
Found opt bandwidth for class, feature0,1
Start to find opt bandwidth for class, feature0,2
Found opt bandwidth for class, feature0,2
Start to find opt bandwidth for class, feature0,3
Found opt bandwidth for class, feature0,3
Start to find opt bandwidth for class, feature0,4
Found opt bandwidth for class, feature0,4
Start to find opt bandwidth for class, feature0,5
Found opt bandwidth for class, feature0,5
Start to find opt bandwidth for class, feature0,6
Found opt bandwidth for class, feature0,6
Start to find opt bandwidth for class, feature0,7
Found opt bandwidth for class, feature0,7
Start to find opt bandwidth for class, feature1,0
Found opt bandwidth for class, feature1,0
Start to find opt bandwidth for class, feature1,1
Found opt bandwidth for class, feature1,1
Start to find opt bandwidth for class, feature1,2
Found opt bandwidth for class,

In [16]:
if checker.feature_val != None:
    checker.get_discrete_posterior()
    print("Got discrete posterior")
checker.clustering()
print("Finished clustering")

Finished clustering


In [17]:
prob_distribution = checker.test_prob_dist(x_test)
prediction = {i: max(prob_distribution[i].items(), key=operator.itemgetter(1))[0] for i in prob_distribution.keys()}
utils.get_accuracy(prediction, y_test)

/Users/pengcheng/Desktop/FYP/Improvement-to-Naive-Bayes-Classifier/utils.py:160: RuntimeWarning: invalid value encountered in matmul
  b = np.matmul(vec, mid_mat)
/Users/pengcheng/Desktop/FYP/Improvement-to-Naive-Bayes-Classifier/utils.py:161: RuntimeWarning: invalid value encountered in matmul
  c = np.matmul(b, vec.T)
/Users/pengcheng/Desktop/FYP/Improvement-to-Naive-Bayes-Classifier/GaussianCopula.py:127: RuntimeWarning: divide by zero encountered in log
  density = sum(marginal_density) + np.log(utils.copula_func(marginal_cdf, R))


0.8251748251748252